In [1]:
import pandas as pd
import numpy as np
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import tensorflow_federated as tff

import nest_asyncio
nest_asyncio.apply()

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

In [2]:
screens = ['Focus', 'Mathisis', 'Memoria', 'Reacton', 'Speedy']
screens_code = ['1', '2', '3', '4', '5']

base_path = "C:/Users/SouthSystem/Federated Learning/DataBioCom/data"
phone_accel_file_paths = []
phone_gyro_file_paths = []

for directories, subdirectories, files in os.walk(base_path):
    for filename in files:
        if "accel" in filename:
            phone_accel_file_paths.append(f"{base_path}/accel/{filename}")
            
data = pd.concat(map(pd.read_csv, phone_accel_file_paths))

In [3]:
data

,x_accel,y_accel,z_accel,screen,player_id,timestamp
0,-0.146046,0.802520,0.586626,MemoriaGame - 1.1.1,06mdn3c,1536502712738
1,-0.031261,0.766375,0.666243,MemoriaGame - 1.1.1,06mdn3c,1536502712738
2,-0.042495,0.769794,0.647682,MemoriaGame - 1.1.1,06mdn3c,1536502712738
3,-0.025888,0.766375,0.647194,MemoriaGame - 1.1.1,06mdn3c,1536502712738
4,-0.028818,0.760025,0.643286,MemoriaGame - 1.1.1,06mdn3c,1536502712738
...,...,...,...,...,...,...
466372,-0.038849,-0.608170,-0.793686,FocusGame - 4.1.1,x8rbf3x,1547393235756
466373,-0.042145,-0.613846,-0.786880,FocusGame - 4.1.1,x8rbf3x,1547393235756
466374,-0.043121,-0.613129,-0.786621,FocusGame - 4.1.1,x8rbf3x,1547393235756
466375,-0.042694,-0.608475,-0.789032,FocusGame - 4.1.1,x8rbf3x,1547393235756


In [4]:
users = data['player_id'].unique()

In [5]:
def normalize_rows(df):
    array = df.values    
    # array = df
    nsamples, nfeatures = array.shape
    nfeatures = nfeatures - 1
    X = array[:, 0:nfeatures]
    y = array[:, -1]
    
    rows, cols = X.shape
    
    for i in range(0, rows):
        row = X[i,:]
        mu = np.mean( row )
        sigma = np.std( row )
        if( sigma == 0 ):
            sigma = 0.0001
        X[i,:] = (X[i,:] - mu) / sigma
            
    df = pd.DataFrame( X )
    df['user'] = y 
    return df

def unique(list1):       
    list_set = set(list1) 
    unique_list = (list(list_set)) 
    unique_list.sort()
    return unique_list

def create_userids( df ):
    array = df.values
    y = array[:, -1]
    return unique( y )

In [6]:
def load_data_1():
    user_list = []
    frame_size = 500
    step = 50
    train = np.empty((0, frame_size, 3))

    for user in users:
        data_user = data[data['player_id']==user]  
        data_user = data_user.iloc[:,[0,1,2]]
        data_user = data_user[500:-500]
        data_user = data_user.values
        data_user = data_user.astype('float32')
        frames = [data_user[i:i+frame_size, :] for i in range(0,data_user.shape[0]-frame_size,step)]
        user_list.extend([user]*len(frames))           
        frames = np.dstack(frames)
        frames = np.rollaxis(frames,-1)
        train = np.vstack((train, frames))
        

    return train, user_list

def load_data_2():
    user_list = []
    train = []
    frame_size = 128
    step = 50

    for user in users:
        data_user = data[data['player_id']==user]  
        data_user = data_user.iloc[:,[0,1,2]]
        for w in range(0, data_user.shape[0] - frame_size, step):
            end = w + frame_size        
            frame = data_user.iloc[w:end,[0, 1, 2]]        
            train.append(frame)
            user_list.append(user)

    return train, user_list

In [7]:
def load_data_3():
    data['session'] = data['player_id'] + "_" + data['timestamp'].apply(str)
    
    counts = data['session'].value_counts()
    counts = counts[counts >= 1]
    counts_list = list(counts.keys())
    df = data[data.session.isin(counts_list) == True]
    
    for idx, val in enumerate(screens):
        df.loc[df.screen.str.contains(screens[idx]), 'screen'] = screens_code[idx]
        
    win_count = 0
    total_win_count = 0
    range_screen = range(1, 6)
    raw_signal = df
    axis_list = ['x_accel', 'y_accel', 'z_accel']
    user_list = []
    window_size = 128
    axis_dict = {}

    for axis in axis_list:  
        features_one = []
        for class_label in range_screen:   
            screen_ID = screens_code[class_label - 1]    
            raw_data_one_activity = np.array(raw_signal.loc[raw_signal['screen'] == screen_ID, [axis]])
            raw_data_one_activity = pd.DataFrame(raw_data_one_activity)   
            player_id_data = np.array(raw_signal.loc[raw_signal['screen'] == screen_ID, ['player_id']])
            player_id_data = pd.DataFrame(player_id_data)  

            for data_point in range(0, len(raw_data_one_activity), window_size):        
                win_count += 1
                start = data_point
                end = start + window_size
                time_domain_window = raw_data_one_activity[start:end] 

                if (len(time_domain_window) == 128):                
                    features_one.append(time_domain_window)
                    if (axis == 'z_accel'):                    
                        user_list.append(player_id_data[start:end][0].unique()[0])                    

        axis_dict[axis] = features_one
        
    new = (axis_dict[axis_list[0]], axis_dict[axis_list[1]], axis_dict[axis_list[2]])
    
    new_x = new[0]    
    new_x = np.array([np.array(x) for x in new_x])
    new_x = new_x.reshape(new_x.shape[0],-1)
    print(new_x.shape)
    
    new_y = new[1]
    new_y = np.array([np.array(x) for x in new_y])  
    new_y = new_y.reshape(new_y.shape[0],-1)
    print(new_y.shape)

    new_z = new[2]
    new_z = np.array([np.array(x) for x in new_z])  
    new_z = new_z.reshape(new_z.shape[0],-1)
    print(new_z.shape)
    
    data_join = pd.DataFrame(np.concatenate((new_x, new_y, new_z), axis=1))
    data_join['user'] = user_list
    
    return data_join

In [7]:
train_set_3 = load_data_3()

(31357, 128)
(31357, 128)
(31357, 128)


In [8]:
train_set_3

,0,1,2,3,4,5,6,7,8,9,...,375,376,377,378,379,380,381,382,383,user
0,-0.006350,0.001954,0.002442,0.001465,-0.004396,-0.010257,-0.001954,0.008792,0.006350,-0.008304,...,0.867484,0.850877,0.835735,0.837200,0.832315,0.830362,0.609095,0.806428,0.832804,06mdn3c
1,-0.049822,-0.054706,-0.033703,-0.025888,-0.017096,-0.021492,-0.046891,-0.011234,0.004396,0.109412,...,0.773702,0.799589,0.766375,0.779563,0.774679,0.788355,0.816197,0.907537,0.810824,06mdn3c
2,0.038587,-0.091828,-0.264739,-0.301861,-0.128462,-0.054218,-0.057637,-0.064475,-0.074244,-0.046891,...,0.769794,0.784936,0.784936,0.785913,0.818639,0.750256,0.798124,0.771748,0.768817,06mdn3c
3,-0.041030,-0.041030,-0.046403,-0.046403,-0.070825,-0.070825,-0.060079,-0.056172,-0.040053,-0.050799,...,0.793728,0.818150,0.811801,0.809358,0.818639,0.823035,0.820104,0.826454,0.790309,06mdn3c
4,-0.010257,-0.060568,-0.039564,-0.033703,-0.025888,-0.022957,-0.031261,-0.031261,-0.047379,-0.040053,...,0.716553,0.868949,0.783471,0.837688,0.769306,0.782005,0.767352,0.758560,0.781028,06mdn3c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31352,-0.087982,0.024231,-0.054306,-0.029510,-0.032318,-0.021622,-0.025528,-0.040161,-0.008102,-0.015244,...,-0.713440,-0.397873,-0.779297,-0.775192,-0.621613,-0.775940,-0.740738,-0.732880,-0.670929,x8rbf3x
31353,-0.212509,-0.056503,-0.258072,-0.054901,-0.095520,-0.128067,-0.076050,-0.077179,-0.097610,-0.086761,...,-0.745773,-0.706970,-0.692856,-0.694855,-0.715042,-0.856445,-0.751465,-0.854660,-0.361664,x8rbf3x
31354,0.150406,0.028015,-0.100739,-0.136841,-0.008072,-0.070007,-0.162521,-0.069672,-0.120972,-0.166962,...,-0.845154,-1.011536,-0.916336,-0.913208,-1.038147,-0.893433,-1.014160,-0.892410,-0.915115,x8rbf3x
31355,0.007492,-0.118988,-0.055145,-0.044754,-0.060135,-0.092682,-0.101700,-0.089127,-0.084930,-0.091431,...,-0.890671,-0.897583,-1.032913,-0.385971,-0.926208,-0.726227,-0.895981,-0.910370,-0.772766,x8rbf3x


In [8]:
train_set_1, user_list1 = load_data_1()
train_set_join_1 = train_set_1.reshape(train_set_1.shape[0], 1500)
data_join = pd.DataFrame(train_set_join_1)
data_join['user'] = user_list1
data_join.shape

(79395, 1501)

In [9]:
train_set_2, user_list2 = load_data_2()

In [10]:
train_set_2 = np.array([np.array(x) for x in train_set_2])  

In [11]:
train_set_join_2 = train_set_2.reshape(train_set_2.shape[0], 384)

In [12]:
train_set_join_2.shape

(80220, 384)

In [13]:
data_join = pd.DataFrame(train_set_join_2)
data_join['user'] = user_list2
data_join.shape

(80220, 385)

In [14]:
df = data_join

In [15]:
df

,0,1,2,3,4,5,6,7,8,9,...,375,376,377,378,379,380,381,382,383,user
0,-0.146046,0.802520,0.586626,-0.031261,0.766375,0.666243,-0.042495,0.769794,0.647682,-0.025888,...,0.005373,0.760025,0.651101,-0.006350,0.763444,0.663312,-0.015142,0.773213,0.661847,06mdn3c
1,0.000000,0.771259,0.675035,0.007327,0.811312,0.539247,-0.040541,0.866995,0.520686,-0.059102,...,-0.042983,0.786890,0.623260,-0.004884,0.777609,0.630098,-0.108435,0.807893,0.972500,06mdn3c
2,0.023446,0.778098,0.655009,0.005861,0.768329,0.650613,0.011723,0.773702,0.648659,0.007327,...,0.013677,0.621306,0.786401,0.014653,0.626679,0.783959,0.021492,0.619841,0.780052,06mdn3c
3,-0.003419,0.760025,0.643775,-0.005861,0.762956,0.662336,-0.003908,0.759537,0.664778,0.000488,...,0.042495,0.703365,0.703365,0.057637,0.652078,0.743418,0.050310,0.635471,0.778586,06mdn3c
4,0.033214,0.625702,0.784448,0.035168,0.613491,0.789820,0.040053,0.615445,0.783959,0.034680,...,0.022469,0.681874,0.725345,0.020026,0.682362,0.750256,0.020026,0.675524,0.740976,06mdn3c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80215,-0.043182,-0.609863,-0.767822,-0.044266,-0.609787,-0.778061,-0.041946,-0.607758,-0.787079,-0.039627,...,-0.043823,-0.607330,-0.780472,-0.045563,-0.607315,-0.783386,-0.040222,-0.604111,-0.783401,x8rbf3x
80216,-0.040771,-0.604309,-0.789261,-0.042007,-0.603592,-0.789734,-0.040344,-0.602325,-0.793396,-0.039627,...,-0.036362,-0.609390,-0.790054,-0.037643,-0.607178,-0.793930,-0.040359,-0.603043,-0.794861,x8rbf3x
80217,-0.039551,-0.605042,-0.789276,-0.039154,-0.607315,-0.777832,-0.040588,-0.605408,-0.774155,-0.043030,...,-0.050720,-0.609039,-0.787018,-0.053665,-0.606857,-0.786743,-0.051178,-0.609070,-0.783844,x8rbf3x
80218,-0.047516,-0.608307,-0.784119,-0.048035,-0.610962,-0.787308,-0.047577,-0.609482,-0.789734,-0.045151,...,-0.041763,-0.605042,-0.790970,-0.043243,-0.605270,-0.792664,-0.045898,-0.604340,-0.789459,x8rbf3x


In [9]:
df = normalize_rows(train_set_3)

In [16]:
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess(df):
    
    userids = create_userids( df )
    nbclasses = len(userids)    
    array = df.values
    nsamples, nfeatures = array.shape
    nfeatures = nfeatures -1 
    X = array[:,0:nfeatures]
    y = array[:,-1]
    
    print(y)
    
    enc = LabelEncoder()
    y = enc.fit_transform(y.reshape(-1,1))
    X = X.reshape(-1, 128, 3)
    
    print(y)

#     def batch_format_fn(element):
#         """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
#         return collections.OrderedDict(
#             x=tf.reshape(element['pixels'], [-1, 784]),
#             y=tf.reshape(element['label'], [-1, 1]))

#     return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [49]:
preprocess(df)

['06mdn3c' '06mdn3c' '06mdn3c' ... 'x8rbf3x' 'x8rbf3x' 'x8rbf3x']
[ 0  0  0 ... 29 29 29]


In [17]:
userids = create_userids( df )
nbclasses = len(userids)

In [18]:
# Fit and evaluate a model
def evaluate_model(df, input_shape, num_filters = 128):
    RANDOM_STATE = 11235
    
    userids = create_userids( df )
    nbclasses = len(userids)
    print(nbclasses)
    array = df.values
    nsamples, nfeatures = array.shape
    nfeatures = nfeatures -1 
    X = array[:,0:nfeatures]
    y = array[:,-1]
    
    enc = OneHotEncoder()
    enc.fit(y.reshape(-1,1))
    y = enc.transform(y.reshape(-1, 1)).toarray()
    X = X.reshape(-1, 128, 3)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)
    
    print(X_train.shape)
    print(X_test.shape)
    print(X_val.shape)
    
    mini_batch_size = int(min(X_train.shape[0]/10, 16))   
    
    # set epochs and batch_size to 1 each due to its purpose solely as example and limiting resource
    # set verbose to 1 to see training progress
    verbose, epochs, batch_size = 1, 1, 1
    EPOCHS = 100
    
    input_layer = keras.layers.Input(input_shape) 

    conv1 = keras.layers.Conv1D(filters=num_filters, kernel_size=8, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)

    conv2 = keras.layers.Conv1D(filters=2*num_filters, kernel_size=5, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)

    conv3 = keras.layers.Conv1D(num_filters, kernel_size=3,padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)

    gap_layer = keras.layers.GlobalAveragePooling1D()(conv3)
    output_layer = keras.layers.Dense(nbclasses, activation='softmax')(gap_layer)
    
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    
    learning_rate = 0.0001
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, 
                                                  min_lr=learning_rate)
    
    cb = [reduce_lr]
    
    # Set precision and recall to calculate F1 score
    precision = tf.keras.metrics.Precision(name='precision')
    recall = tf.keras.metrics.Recall(name='recall')
    
    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), metrics=['accuracy', precision, recall]) 
    
    model.summary()

    X_train = np.asarray(X_train).astype(np.float32)
    X_val = np.asarray(X_val).astype(np.float32)
    
    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))

    BATCH_SIZE = mini_batch_size
    SHUFFLE_BUFFER_SIZE = 100
    
    train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
    val_ds = val_ds.batch(BATCH_SIZE)
    
    hist = model.fit(train_ds, 
                      epochs=EPOCHS,
                      verbose=True, 
                      validation_data=val_ds, 
                      callbacks=cb)
    
    hist_df = pd.DataFrame(hist.history) 
    
    print(hist_df)
    
    # get evaluation metrics
    # categorical_accuracy = hist.history['categorical_accuracy'][epochs-1]
    # accuracy = hist.history['accuracy'][epochs-1]
    precision = hist.history['precision'][epochs-1]
    recall = hist.history['recall'][epochs-1]
    
    X_test = np.asarray(X_test).astype(np.float32)    
    y_true = np.argmax( y_test, axis=1)
    y_pred = np.argmax( model.predict(X_test), axis=1)
    accuracy = accuracy_score(y_true, y_pred) 
    
    return accuracy, precision, recall

# summarize scores
def summarize_results(scores, f1):
    m, s = np.mean(scores), np.std(scores)
    m2, s2 = np.mean(f1), np.std(f1)
    print('Accuracy: %.3f%% (+/-%.3f), F1 score: %.3f%% (+/-%.3f)' % (m, s, m2, s2))
    
# run an experiment
def run_experiment():
    accuracy, precision, recall = evaluate_model(df, (128, 3))
    print(accuracy)
    accuracy = accuracy * 100.0
    f1_score = (2.0*((precision * recall)/(precision + recall))) * 100.0

In [ ]:
run_experiment()

In [19]:
def batch_data(data_shard):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))

    BATCH_SIZE = 30
    SHUFFLE_BUFFER_SIZE = 100
    
    return dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

In [20]:
keras_model = create_keras_model_seq((128,3))
keras_model.summary()

NameError: name 'create_keras_model_seq' is not defined

In [21]:
def load_client_dataset(df):
    RANDOM_STATE = 11235
    userids = create_userids( df )
    nbclasses = len(userids)
    array = df.values
    nsamples, nfeatures = array.shape
    nfeatures = nfeatures -1 
    X = array[:,0:nfeatures]
    y = array[:,-1]
        
    enc = OneHotEncoder()
    enc.fit(y.reshape(-1,1))
    y = enc.transform(y.reshape(-1, 1)).toarray()
    X = X.reshape(-1, 128, 3)
    
    print(y.shape)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)
    
    
    mini_batch_size = int(min(X_train.shape[0]/10, 16))   

    X_train = np.asarray(X_train).astype(np.float32)
    X_val = np.asarray(X_val).astype(np.float32)
    
    return create_clients(X_train, y_train, num_clients=10, initial='client')

In [24]:
clients = load_client_dataset(df)

(80220, 30)


In [31]:
def create_keras_model_seq(input_shape, num_filters = 128):
    model = Sequential()
    model.add(keras.layers.Conv1D(filters=num_filters, kernel_size=8, padding='same', activation='relu', input_shape=input_shape))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Conv1D(filters=2*num_filters, kernel_size=5, padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Conv1D(num_filters, kernel_size=3,padding='same', activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Dense(nbclasses, activation='softmax'))
    
    return model

In [29]:
def model_fn():
    keras_model = create_keras_model_seq((128,3))
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=clients_batched[0].element_spec,
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=[tf.keras.metrics.CategoricalAccuracy(name='acc'),\
               tf.keras.metrics.Precision(name='pr'),\
               tf.keras.metrics.Recall(name='rc')\
              ])

In [25]:
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

In [26]:
clients_batched = list(clients_batched.values())

In [27]:
clients_batched[0].element_spec

(TensorSpec(shape=(None, 128, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 30), dtype=tf.float64, name=None))

In [32]:
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(0.0001))

# Init
state = trainer.initialize()

# Simulate "batch" learning
for i in range(100):
    state, metrics = trainer.next(state, clients_batched)
    print('> Iteration #%d | Accuracy: %.3f%%, F1 score: %.3f%%' % ( \
            i+1, \
            metrics['train']['acc'] * 100.0, \
            2.0*((metrics['train']['pr'] * metrics['train']['rc'])/(metrics['train']['pr'] + metrics['train']['rc'])) * 100.0 \
         )
    )

> Iteration #1 | Accuracy: 45.870%, F1 score: 31.198%
> Iteration #2 | Accuracy: 50.424%, F1 score: 40.869%
> Iteration #3 | Accuracy: 52.146%, F1 score: 44.170%
> Iteration #4 | Accuracy: 53.605%, F1 score: 46.445%
> Iteration #5 | Accuracy: 54.667%, F1 score: 47.983%
> Iteration #6 | Accuracy: 55.159%, F1 score: 49.686%
> Iteration #7 | Accuracy: 55.816%, F1 score: 50.176%
> Iteration #8 | Accuracy: 56.194%, F1 score: 50.849%
> Iteration #9 | Accuracy: 56.971%, F1 score: 52.164%
> Iteration #10 | Accuracy: 57.550%, F1 score: 53.070%
> Iteration #11 | Accuracy: 57.769%, F1 score: 53.310%
> Iteration #12 | Accuracy: 58.194%, F1 score: 53.719%
> Iteration #13 | Accuracy: 59.125%, F1 score: 54.894%
> Iteration #14 | Accuracy: 59.404%, F1 score: 55.431%
> Iteration #15 | Accuracy: 59.904%, F1 score: 55.865%
> Iteration #16 | Accuracy: 60.272%, F1 score: 56.518%
> Iteration #17 | Accuracy: 60.740%, F1 score: 57.235%
> Iteration #18 | Accuracy: 60.920%, F1 score: 57.512%
> Iteration #19 | A

In [23]:
def create_clients(data_list, label_list, num_clients=10, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(data_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))}

In [ ]:
def build_fcn(input_shape, nb_classes, file_path, num_filters = 128):
    input_layer = keras.layers.Input(input_shape) 

    conv1 = keras.layers.Conv1D(filters=num_filters, kernel_size=8, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)

    conv2 = keras.layers.Conv1D(filters=2*num_filters, kernel_size=5, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)

    conv3 = keras.layers.Conv1D(num_filters, kernel_size=3,padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)

    gap_layer = keras.layers.GlobalAveragePooling1D()(conv3)
    output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), metrics=['categorical_accuracy'])
    learning_rate = 0.0001
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, 
                                                  min_lr=learning_rate)
    
    model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='val_loss', save_best_only=True, verbose=1)

    return model

In [ ]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad

def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [ ]:
clients = create_clients(X_train, y_train, num_clients=10, initial='client')

In [ ]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

In [ ]:
global_model = build_fcn((128, 3), nbclasses, "foo.h5")
comms_round = 100

In [ ]:
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:    
        local_model = build_fcn((128, 3), nbclasses, "foo.h5")
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)